# Training Regression - Reaction

# Import packages

In [ ]:
import os
import sys

current_path=os.getcwd()
print(current_path)

parent_path=os.path.dirname(current_path)
print(parent_path)

if parent_path not in sys.path:
    sys.path.append(parent_path)

In [ ]:
import pandas as pd
from lightning import pytorch as pl
from pathlib import Path

from chemprop import data, featurizers, models, nn

# Change data inputs here

## Load data

In [ ]:
import numpy as np
chemprop_dir = Path.cwd().parent
num_workers = 0  # number of workers for dataloader. 0 means using main process for data loading
# smiles_column = 'AAM'
# target_columns = ['lograte']

## Perform data splitting for training, validation, and testing

## Get ReactionDatasets

In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch

# --- BƯỚC 1: ĐỊNH NGHĨA DATASET CHO PRE-TRAINING ---

class MaskedFeatureDataset(Dataset):
    """
    Dataset này sẽ:
    1. Tải các features từ file NPZ.
    2. Trong mỗi lần lấy dữ liệu (__getitem__), nó sẽ che ngẫu nhiên một phần 
       của node_attrs và edge_attrs.
    3. Trả về cả dữ liệu gốc và dữ liệu đã bị che.
    """
    def __init__(self, node_attrs, edge_attrs, edge_indices, mask_fraction=0.15):
        self.node_attrs = [torch.tensor(attrs, dtype=torch.float32) for attrs in node_attrs]
        self.edge_attrs = [torch.tensor(attrs, dtype=torch.float32) for attrs in edge_attrs]
        self.edge_indices = [torch.tensor(idx, dtype=torch.long) for idx in edge_indices]
        self.mask_fraction = mask_fraction

    def __len__(self):
        return len(self.node_attrs)

    def __getitem__(self, idx):
        # Lấy dữ liệu gốc
        original_nodes = self.node_attrs[idx]
        original_edges = self.edge_attrs[idx]
        edge_index = self.edge_indices[idx]

        # Tạo bản sao để che
        masked_nodes = original_nodes.clone()
        masked_edges = original_edges.clone()

        # Che ngẫu nhiên một phần node features
        num_node_features_to_mask = int(original_nodes.shape[0] * self.mask_fraction)
        node_mask_indices = torch.randperm(original_nodes.shape[0])[:num_node_features_to_mask]
        masked_nodes[node_mask_indices] = 0.0 # Che bằng cách gán giá trị 0

        # Che ngẫu nhiên một phần edge features
        num_edge_features_to_mask = int(original_edges.shape[0] * self.mask_fraction)
        edge_mask_indices = torch.randperm(original_edges.shape[0])[:num_edge_features_to_mask]
        masked_edges[edge_mask_indices] = 0.0 # Che bằng cách gán giá trị 0
        
        return {
            "masked_nodes": masked_nodes,
            "masked_edges": masked_edges,
            "original_nodes": original_nodes,
            "original_edges": original_edges,
            "node_mask_indices": node_mask_indices,
            "edge_mask_indices": edge_mask_indices,
            "edge_index": edge_index
        }

# --- TẢI VÀ KẾT HỢP TẤT CẢ DỮ LIỆU ĐỂ PRE-TRAIN ---
# Pre-training là tự giám sát nên chúng ta có thể dùng cả train/val/test data

# Tải dữ liệu từ các file NPZ của bạn
train_npz = np.load(f'../chemprop/data/RC/full/barriers_rdb7/barriers_rdb7_aam_train_rc_processed_data.npz', allow_pickle=True)
val_npz = np.load(f'../chemprop/data/RC/full/barriers_rdb7/barriers_rdb7_aam_val_rc_processed_data.npz', allow_pickle=True)
test_npz = np.load(f'../chemprop/data/RC/full/barriers_rdb7/barriers_rdb7_aam_test_rc_processed_data.npz', allow_pickle=True)

# Kết hợp dữ liệu
all_node_attrs = np.concatenate((train_npz['node_attrs'], val_npz['node_attrs'], test_npz['node_attrs']))
all_edge_attrs = np.concatenate((train_npz['edge_attrs'], val_npz['edge_attrs'], test_npz['edge_attrs']))
all_edge_indices = np.concatenate((train_npz['edge_indices'], val_npz['edge_indices'], test_npz['edge_indices']))

print(f"Tổng số mẫu để pre-train: {len(all_node_attrs)}")

def get_reverse_edge_index(edge_index):
    """Tính toán chỉ số của các cạnh ngược."""
    rev_edge_index = torch.zeros_like(edge_index[0])
    for i in range(edge_index.shape[1]):
        # Tìm cạnh ngược (j, i) cho mỗi cạnh (i, j)
        edge_to_find = edge_index[:, i].flip(0)
        # So sánh để tìm vị trí
        matches = (edge_index.T == edge_to_find).all(dim=1)
        # Lấy chỉ số đầu tiên tìm thấy
        rev_idx = torch.where(matches)[0]
        if rev_idx.numel() > 0:
            rev_edge_index[i] = rev_idx[0]
        else:
            # Xử lý trường hợp không tìm thấy cạnh ngược (ít khả năng xảy ra nếu đồ thị là vô hướng)
            rev_edge_index[i] = -1 # hoặc một giá trị đặc biệt
    return rev_edge_index


def collate_pretraining_batch(samples):
    """
    Hàm này sẽ gộp một list các sample (dictionaries) thành một batch duy nhất,
    đồng thời đảm bảo các thuộc tính V, E, và rev_edge_index được đặt tên đúng.
    """
    batch_data = []
    for i, sample in enumerate(samples):
        edge_index = sample["edge_index"]
        
        # *** THÊM BƯỚC NÀY ***
        # Tính toán rev_edge_index
        rev_edge_index = get_reverse_edge_index(edge_index)

        data_point = Data(
            V=sample["masked_nodes"],
            E=sample["masked_edges"],
            edge_index=edge_index,
            
            # Thêm rev_edge_index vào đối tượng Data
            rev_edge_index=rev_edge_index,
            
            original_V=sample["original_nodes"],
            node_mask_indices=sample["node_mask_indices"],
            sample_idx=i
        )
        batch_data.append(data_point)

    return Batch.from_data_list(batch_data)

# Khởi tạo lại DataLoader với collate_fn mới
pretrain_dataset = MaskedFeatureDataset(all_node_attrs, all_edge_attrs, all_edge_indices)
pretrain_loader = DataLoader(pretrain_dataset, batch_size=32, shuffle=True, collate_fn=collate_pretraining_batch)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message passing

Message passing blocks must be given the shape of the featurizer's outputs.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

## Construct MPNN

In [ ]:
# SỬA LẠI LỚP PRETRAININGMPNN

from torch.nn import Linear, MSELoss 

from chemprop.data import BatchMolGraph
from chemprop.nn import MessagePassing
from chemprop.schedulers import build_NoamLike_LRSched

class PretrainingMPNN(pl.LightningModule):
    def __init__(self, message_passing, node_feature_dim):
        super().__init__()
        self.message_passing = message_passing
        hidden_dim = self.message_passing.output_dim
        self.node_prediction_head = Linear(hidden_dim, node_feature_dim)
        self.loss_fn = MSELoss()
        

# Sửa trong lớp PretrainingMPNN

    def forward(self, batch):
        # Hàm forward của BondMessagePassing sẽ tự động dùng batch.V và batch.E
        
        # SỬA Ở ĐÂY: Gán kết quả cho một biến duy nhất
        node_embeddings = self.message_passing(batch)
        
        predicted_node_features = self.node_prediction_head(node_embeddings)
        return predicted_node_features

    def training_step(self, batch, batch_idx):
            predicted_nodes = self.forward(batch)
            
            # Tạo boolean mask cho toàn bộ batch
            total_nodes = batch.V.shape[0] # Dùng batch.V thay vì batch.x
            node_mask = torch.zeros(total_nodes, dtype=torch.bool, device=self.device)

            base_idx = 0
            for i in range(batch.num_graphs):
                # Lấy các chỉ số mask cho từng đồ thị con trong batch
                sample_mask_indices = batch.node_mask_indices[batch.batch == i]
                
                # Cập nhật mask tổng
                node_mask[base_idx + sample_mask_indices] = True
                
                # Di chuyển đến điểm bắt đầu của đồ thị tiếp theo
                num_nodes_in_sample = (batch.batch == i).sum()
                base_idx += num_nodes_in_sample
            
            # Chỉ tính loss trên các node đã bị che
            if node_mask.sum() > 0:
                # Dùng batch.original_V để lấy feature gốc
                loss = self.loss_fn(predicted_nodes[node_mask], batch.original_V[node_mask])
                self.log("train_loss", loss, prog_bar=True)
                return loss
                
            return None

    def configure_optimizers(self):
            return torch.optim.Adam(self.parameters(), lr=1e-3)

# Training and testing

## Test results

In [ ]:
# --- BƯỚC 3: KHỞI TẠO VÀ HUẤN LUYỆN MÔ HÌNH PRE-TRAINING (ĐÃ SỬA LỖI) ---

# Lấy thông số chiều từ dữ liệu
node_feature_dim = pretrain_dataset.node_attrs[0].shape[1]
edge_feature_dim = pretrain_dataset.edge_attrs[0].shape[1]
fdims = (node_feature_dim, edge_feature_dim)

# Khởi tạo encoder D-MPNN
mp_encoder = nn.BondMessagePassing(*fdims)

# Khởi tạo mô hình pre-training
# SỬA Ở ĐÂY: Xóa `edge_feature_dim` khỏi lời gọi hàm
pretrain_model = PretrainingMPNN(
    message_passing=mp_encoder, 
    node_feature_dim=node_feature_dim
)

# Khởi tạo Trainer của PyTorch Lightning
trainer = pl.Trainer(
    max_epochs=200, 
    accelerator="auto",
    devices=1,
    enable_progress_bar=True,
    logger=True
)

# Bắt đầu pre-training!
print("Bắt đầu Pre-training...")
trainer.fit(pretrain_model, pretrain_loader)
print("Pre-training hoàn tất!")


# --- BƯỚC 4: LƯU LẠI ENCODER ĐÃ ĐƯỢC HUẤN LUYỆN ---
output_path = "pretrained_dmpnn_encoder.pt"
torch.save(pretrain_model.message_passing.state_dict(), output_path)
print(f"Encoder đã pre-train được lưu tại: {output_path}")